In [4]:
import numpy as np
import cv2


face_cascade = cv2.CascadeClassifier('data\\xml\\haarcascade_frontalface_default.xml')
mouth_cascade = cv2.CascadeClassifier('data\\xml\\haarcascade_mcs_mouth.xml')


# Menentukan batas threshold manual; diambil nilai tengah (255/2)
bw_threshold = 127

# Pesan user
font = cv2.FONT_HERSHEY_SIMPLEX
org = (50, 50)
weared_mask_font_color = (255, 255, 255)
not_weared_mask_font_color = (0, 0, 255)
thickness = 2
font_scale = 0.5
weared_mask = "TERIMA KASIH SUDAH MEMAKAI MASKER"
not_weared_mask = "TOLONG PAKAI MASKER ANDA"

# Read video
cap = cv2.VideoCapture(0)

while 1:
    # Get individual frame
    # Menggunakan cv2.flip() method
    # Menggunakan Flip code 0 untuk flip horizontal
    ret, img = cap.read()
    img = cv2.flip(img,1)

    # Konversi citra RGB ke Gray
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Convert image in black and white
    (thresh, black_and_white) = cv2.threshold(gray, bw_threshold, 255, cv2.THRESH_BINARY)
    #cv2.imshow('black_and_white', black_and_white)

    # Dalam klasifikasi ini digunakan modul detectMultiScale
    # Fungsi tersebut akan mengembalikan persegi panjang dengn koordinat (x, y, w, h) 
    # dan akan mengelilingi bagian yang diinginkan
    
    # Deteksi wajah
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Face prediction for black and white
    faces_bw = face_cascade.detectMultiScale(black_and_white, 1.1, 4)


    if(len(faces) == 0 and len(faces_bw) == 0):
        cv2.putText(img, "Wajah tidak ditemukan...", org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
    elif(len(faces) == 0 and len(faces_bw) == 1):
        # Untuk masker putih yang menutupi mulut, dengan prediksi wajah gambar abu-abu(gray) tidak terjadi
        cv2.putText(img, weared_mask, org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
    else:
        # Draw rectangle (area persegi panjang)
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), 2)
            roi_gray = gray[y:y + h, x:x + w]
            roi_color = img[y:y + h, x:x + w]


            # Deteksi mulut/bibir
            mouth_rects = mouth_cascade.detectMultiScale(gray, 1.5, 5)

        # Wajah terdeteksi tetapi mulut/bibir tidak terdeteksi berarti menandakan memakai masker
        if(len(mouth_rects) == 0):
            cv2.putText(img, weared_mask, org, font, font_scale, weared_mask_font_color, thickness, cv2.LINE_AA)
        else:
            for (mx, my, mw, mh) in mouth_rects:

                if(y < my < y + h):
                    # Wajah dan mulut terdeteksi tetapi koordinat bibir berada di dalam koordinat wajah yang menandakan
                    # tidak memakai masker
                    cv2.putText(img, not_weared_mask, org, font, font_scale, not_weared_mask_font_color, thickness, cv2.LINE_AA)
                    break

    # Output
    cv2.imshow('Mask Detection', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

# Release video
cap.release()
cv2.destroyAllWindows()
